In [ ]:
import numpy as np
from skimage import io

from matplotlib import pyplot as plt

from matplotlib import cm
import networkx as nx
from tqdm.notebook import tqdm

from bresenham import bresenham
from matplotlib import cm, colors
import matplotlib.patches as mpatches
from collections import Counter
import pandas as pd

import matplotlib.pyplot as plt

import itertools

from mpire import WorkerPool

import shutil
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pathlib import Path
import json

from wc_cv.src.utils import grainPreprocess, grainMark, SEMDataset, grainDraw, Crack

from pathos.multiprocessing import ProcessingPool as Pool  # Alternative parallelization
import copy

import graph_tool.all as gt
from itertools import islice

In [ ]:
eps = 300
# image = io.imread('wc_cv/cv/plots/WC+8Co_5_crack.jpg')[200:-300]
image = io.imread('wc_cv/cv/plots/WC+8Co_5_crack.jpg')[200:-300]
# image = io.imread('wc_cv/cv/plots/WC+8Co_5_fixed_001_cropped.jpg')

border=30

(entry_nodes,
 exit_nodes,
 img_contours,
 img_preprocessed_final,
 cnts,
 nodes_metadata)  = Crack.preprocess_graph_image(image, border=border, disk=5)

plot=False
# plot=True

if plot:
    fig, axes = plt.subplots(1,2,figsize=(20,20))
    axes[0].imshow(img_contours)
    axes[1].imshow(img_preprocessed_final)
    # plt.savefig('miniature_figures_aligned_crack.jpg', bbox_inches='tight')

    # axes[0].set_ylim(1000,1700)
    # axes[1].set_ylim(1000,1700)
    plt.show()

g, img_contours =  Crack.create_crack_graph(img_preprocessed_final.shape, cnts, nodes_metadata, eps=eps)

In [ ]:
save=False
# save=True

Crack.Viz.graph_plot(g, img_preprocessed_final, name='test.jpg', save=save)

In [ ]:
%%time

# entry_nodes = [ 0,3,4, 11]
# exit_nodes = [ 84, 30, 31]

# WC+8Co_5_fixed_001_cropped.jpg
entry_nodes = [ 4, 15, 13, 16, 34, 27]
exit_nodes = [ 527, 528, 519, 522]

# WC+8Co_5_crack
# entry_nodes = [ 4, 11, 16, 61, 62, 66]
# exit_nodes = [ 307, 308, 515, 529]

# edge types
# 0 - Co
# 1 - WC-Co
# 2 - WC
# 3 - WC-WC

co_co_e_max=20
wc_co_e_max=20
first_k_paths=10

parallel=True
# parallel=False
workers=20

energies = Crack.Energy.get_energies(g,
                        cnts,
                        nodes_metadata,
                        entry_nodes,
                        exit_nodes,
                        co_co_e_max,
                        wc_co_e_max,
                        first_k_paths=first_k_paths,
                        parallel=parallel,
                        workers=workers)


In [ ]:
pairs_num=len(entry_nodes)*len(exit_nodes)

tmp_energies=np.zeros((pairs_num, co_co_e_max, wc_co_e_max)).tolist()
path_index=0

for k in range(pairs_num):
    for i,co_co_e in enumerate(tqdm(range(0,co_co_e_max))):
        for j,wc_co_e in enumerate(range(0,wc_co_e_max)):
            tmp_energies[i][j][k]=energies[i][j][k]


In [ ]:
pairs_num

In [ ]:
energies.__len__()

In [ ]:
# name='wc_cv/cv/WC+8Co_5_fixed_001_cropped_optimized_e.jpg'
name='wc_cv/cv/WC+8Co_5_crack_fixed_optimized_e_labeled_no_wc_10.jpg'

# save=True
save=False

path_index=0

Crack.Viz.plot_optimized_energies_seq(energies, N=5,M=5,
                                    co_co_e_max=20,
                                    wc_co_e_max=20,
                                    name=name)

In [ ]:
%%time

# entry_nodes = [ 0,3,4, 11]
# exit_nodes = [ 84, 30, 31]

# WC+8Co_5_fixed_001_cropped.jpg
entry_nodes = [ 4, 15, 13, 16, 34, 27]
exit_nodes = [ 527, 528, 519, 522]

# WC+8Co_5_crack
# entry_nodes = [ 4, 11, 16, 61, 62, 66]
# exit_nodes = [ 307, 308, 515, 529]

# edge types
# 0 - Co
# 1 - WC-Co
# 2 - WC
# 3 - WC-WC

co_co_e_max=20
wc_co_e_max=20
path_index=0
first_k_paths=1

dfs=[]
for i,co_co_e in enumerate(tqdm(range(1,co_co_e_max))):
    dfs.append([])
    for j,wc_co_e in enumerate(range(1,wc_co_e_max)):
        energy = {
                  0: co_co_e, # Co
                  1: wc_co_e, # WC-Co
                  2: 20, # WC
                  } 
        df = Crack.Energy.get_energies(g, cnts, nodes_metadata, entry_nodes, exit_nodes, energy)
        dfs[i].append(df)

In [ ]:
# name='wc_cv/cv/WC+8Co_5_fixed_001_cropped_optimized_e.jpg'
name='wc_cv/cv/WC+8Co_5_crack_optimized_e.jpg'

save=True
# save=False


Crack.Viz.plot_optimized_energies(dfs, N=5,M=5,
                                  co_co_e_max=20,
                                  wc_co_e_max=20,
                                  name=name,
                                  save=save)

# Fixed paths energies 

In [ ]:
def get_paths_energies(G, paths):
    
    all_paths=[]
    all_entry_nodes=[]
    all_exit_nodes=[]
    all_path_len_edges=[]
    all_path_len_pixels=[]
    
    all_wc_edges = []
    all_co_edges = []
    all_wc_co_edges = []

    all_wc_edges_lens = []
    all_co_edges_lens = []
    all_wc_co_edges_lens = []
    
    all_wc_pixels = []
    all_co_pixels = []
    all_wc_co_pixels = []

    for i in range(len(paths)):
        entry_node, exit_node = paths.iloc[i]['entry_node'], paths.iloc[i]['exit_node']
        path =  paths.iloc[i]['path']
        
        all_paths.append(path)
        all_entry_nodes.append(int(entry_node))
        all_exit_nodes.append(int(exit_node))
        all_path_len_edges.append(len(path))
        
        # calculating energy
        l=nx.path_weight(G, path, 'weight')
        all_path_len_pixels.append(l)
        
        points_stack = np.stack([path, np.roll(path,shift=-1,axis=0)],axis=1)[:-1]

        types = []
        edges_lens=[]
        for points in points_stack:
            node1,node2 = points
            edges_lens.append(nx.path_weight(G,[node1,node2], 'weight'))
            edge_type = Crack.get_edge_type(node1,
                                            node2,
                                            cnts,
                                            nodes_metadata)
            types.append(edge_type)
            
        
        types = np.array(types)
        edges_lens = np.array(edges_lens)
        
        wc_edges_indices = np.where(types==0)[0]
        co_edges_indices = np.where(types==2)[0]
        wc_co_edges_indices = np.where(types==1)[0]

        wc_pixels_lens=edges_lens[wc_edges_indices]
        co_pixels_lens=edges_lens[co_edges_indices]
        wc_co_pixels_lens=edges_lens[wc_co_edges_indices]
        z = np.sum(edges_lens)
            
        all_wc_edges.append(len(wc_edges_indices)/len(path))
        all_co_edges.append(len(co_edges_indices)/len(path))
        all_wc_co_edges.append(len(wc_co_edges_indices)/len(path))

        all_wc_edges_lens.append(len(wc_pixels_lens))
        all_co_edges_lens.append(len(co_pixels_lens))
        all_wc_co_edges_lens.append(len(wc_co_pixels_lens))
        
        all_wc_pixels.append(np.sum(wc_pixels_lens)/z)
        all_co_pixels.append(np.sum(co_pixels_lens)/z)
        all_wc_co_pixels.append(np.sum(wc_co_pixels_lens)/z)

    return pd.DataFrame({'path':all_paths,
                         'path_len_edges':all_path_len_edges,
                         'path_len_pixels': np.round(all_path_len_pixels,2),
                         'energy': np.round(all_path_len_pixels,2),
                         
                         'entry_node':all_entry_nodes,
                         'exit_node':all_exit_nodes,
                         
                         'wc_edges': np.round(all_wc_edges,2),
                         'co_edges':np.round( all_co_edges,2),
                         'wc_co_edges': np.round(all_wc_co_edges,2),
                         
                         'wc_pixels': np.round(all_wc_pixels,2), 
                         'co_pixels': np.round(all_co_pixels,2), 
                         'wc_co_pixels': np.round(all_wc_co_pixels,2), 

                         'wc_num': all_wc_edges_lens,
                         'co_num': all_co_edges_lens,
                         'wc_co_num': all_wc_co_edges_lens,
                        })


def get_e_fixed_optimized_paths(g, paths, energy, workers=23):
    
    # modify the energies of edges
    g_weighed = g.copy()
    
    for u, v in g_weighed.edges():
        path_len = g.edges[u,v]['path_len']
        edge_type = g.edges[u,v]['edge_type']
        e=energy[edge_type]
        g_weighed[u][v]['weight'] = path_len*e

    # find the minimal energy paths
    results = get_paths_energies(g_weighed, paths)
    
    df = copy.copy(results)
    
    df['index'] = range(len(df))
    df.set_index('index')
    
    
    return df

In [ ]:
%%time

N=36
workers=23

np.random.seed(51)

# WC+8Co_5_crack
# entry_nodes = [ 4, 11, 16, 61, 62, 66]
# exit_nodes = [ 307, 308, 515, 529]

# WC+8Co_5_fixed_001_cropped.jpg
entry_nodes = [ 4, 15, 13, 16, 34, 27]
exit_nodes = [ 527, 528, 519, 522]

# hand labeled data
entry_nodes = [ 636, 635, 625, 639, ]
exit_nodes = [ 399, 398, 1092]

energy_o = {0: 10, # WC-WC
            1: 7, # WC-Co
            2: 7 # Co-Co
            3: 10, # WC-WC
           } 

g_weighed = g.copy()

for u, v in g_weighed.edges():
    path_len = g.edges[u,v]['path_len']
    edge_type = g.edges[u,v]['edge_type']
    e=energy_o[edge_type]
    g_weighed[u][v]['weight'] = path_len*e


# get all paths
cart_list=[entry_nodes, exit_nodes]
cart_list=[element for element in itertools.product(*cart_list)]
cart_list=[(g_weighed,cnts, nodes_metadata, line[0],line[1]) for line in cart_list]


with WorkerPool(n_jobs=workers) as pool:
    results = pool.map(Crack.Energy.find_shortest_energy_paths, cart_list, progress_bar= True)

dfs_o = pd.concat(results,axis=0)

# paths_indices = np.random.choice(range(len(dfs_o)), N)
# print(paths_indices)
# paths = dfs_o[paths_indices]
paths = copy.copy(dfs_o)

dfs=[]
print(1)
# calculate energy heatmp for nodes cartesian product
for i,co_co_e in enumerate(tqdm(range(1,20))):
    dfs.append([])
    for j,wc_co_e in enumerate(range(1,20)):
        energy = {
                  0: co_co_e, # Co
                  1: wc_co_e, # WC-Co
                  2: 20, # WC
                  3: 10, # WC-WC
                  } 
        df = get_e_fixed_optimized_paths(g, paths, energy)
        dfs[i].append(df)

In [ ]:
energy_grid=[]

# dfs=np.load('dfs_crack_fixed.npy',allow_pickle=True)
# dfs=np.load('dfs_crack.npy',allow_pickle=True)

N,M=5,5

fig, axes = plt.subplots(N, M, figsize=(50, 50))

step=0
for i in tqdm(range(N)):
    for j in range(M):      
        energy_grid=[] 
        if step<len(dfs[0][0]):
            for k,co_co_e in enumerate(range(1,20)):
                energy_grid.append([])
                
                for p,wc_co_e in enumerate(range(1,20)):
                    df = pd.DataFrame(dfs[k][p],columns=[
                                    'path',
                                    'path_len_edges',
                                    'path_len_pixels',
                                    'energy',
                                    'entry_node',
                                    'exit_node',
                                    'wc_edges',
                                    'co_edges',
                                    'wc_co_edges',
                                    'wc_pixels',
                                    'co_pixels',
                                    'wc_co_pixels',
                                    'wc_num',
                                    'co_num',
                                    'wc_co_num',
                                    'index'])
                    val = df['energy'].iloc[step]
                    energy_grid[k].append(val)
            
            data = np.array(energy_grid)
            data = data/np.max(data)
            im = axes[i,j].imshow(data)
            
            axes[i,j].set_title(f'pix {dfs[k][p]["path_len_pixels"].values[step]}, edg { dfs[k][p]["path_len_edges"].values[step] }, entry {dfs[k][p]["entry_node"].values[step]}, exit {dfs[k][p]["exit_node"].values[step]}', fontsize=20)
            
            divider = make_axes_locatable(axes[i,j])
            cax = divider.append_axes('right', size='5%', pad=0.05)
            fig.colorbar(im, cax=cax, orientation='vertical')
            fig.tight_layout()
        
            axes[i,j].invert_yaxis()
            for k,co_co_e in enumerate(range(1,20)):
                for p,wc_co_e in enumerate(range(1,20)):
                    axes[i,j].text(k + 0.5, p + 0.5, '%.2f' % data[p, k],
                     horizontalalignment='center',
                     verticalalignment='center',
                     )
            axes[i,j].set_ylabel(f'co_co_e ({ dfs[k][p]["co_num"].values[step]})',  fontsize=20)
            axes[i,j].set_xlabel(f'wc_co_e ({ dfs[k][p]["wc_co_num"].values[step]})',  fontsize=20)
            step+=1
        else:
            fig.delaxes(axes[i][j])


# plt.savefig('wc_cv/cv/energies_crack_fixed_paths.jpg')
plt.savefig('wc_cv/cv/energies_crack_fixed_paths_fixed.jpg')
plt.show()

# Dont know

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(15,20))
border=30

pos = nx.get_node_attributes(g, 'pos')

df_shortest_entry = copy.copy(df)

x=329
y=33

# df_shortest_entry=df_shortest_entry[df_shortest_entry['path'].apply(lambda arr: x in arr)]
# df_shortest_entry=df_shortest_entry[df_shortest_entry['path'].apply(lambda arr: y in arr)]

cmap = cm.get_cmap("Spectral")
# need to normalize because color maps are defined in [0, 1]
norm = colors.Normalize(df_shortest_entry['path_len_pixel'].values.min(), df_shortest_entry['path_len_pixel'].values.max())


for i in range(len(df_shortest_entry)):
    line = df_shortest_entry.iloc[i]
    path = line['path']
    
    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges,edge_color=cmap(norm(line['path_len_pixel'])),width=2)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['entry_node']],node_color='blue',node_size = 100)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['exit_node']],node_color='red',node_size = 100)


    labels = {line['entry_node']: line['entry_node']}
    nx.draw_networkx_labels(g, pos,labels, font_size=20, font_color='orange')

    labels = {line['exit_node']: line['exit_node']}
    nx.draw_networkx_labels(g, pos,labels, font_size=20,font_color='black')



im = axes.imshow(img_aligned, cmap='gray')

plt.axis("on")
axes.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

axes.arrow(12, border-5, 0, 200, width=0.3, length_includes_head=True, head_width=10, head_length=10,color=(0,0,0))

axes.text(2, img_aligned.shape[0]*2/3,'hit direction',rotation=90, color=(0, 0, 0),fontsize=15)
axes.text(img_aligned.shape[1]/2-50, border/3,'entry nodes', color=(0, 0, 1),fontsize=15)
axes.text(img_aligned.shape[1]/2-25,img_aligned.shape[0] - border/3,'exit nodes', color=(1, 0, 0),fontsize=15)


fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes)

# plt.savefig('shortests_paths_crack.jpg',bbox_inches='tight')

plt.show()

# Shortest paths

In [ ]:
# 'path':all_paths,
# 'path_len_edges':all_path_len_edges,
# 'path_len_pixel':all_path_len_pixels,
# 'entry_node':all_entry_nodes,
# 'exit_node':all_exit_nodes

entry_nodes = np.unique(df['entry_node'])
exit_nodes = np.unique(df['exit_node'])

N=1

shortest_entry_paths = []
for entry_node in tqdm(entry_nodes):
    # for i in range(N):
    row = df[df.entry_node==entry_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_entry_paths.append(row)

shortest_exit_paths = []
for exit_node in tqdm(exit_nodes):
    # for i in range(N):
    row = df[df.exit_node==exit_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_exit_paths.append(row)

df_shortest_entry = pd.DataFrame(shortest_entry_paths)
df_shortest_exit = pd.DataFrame(shortest_exit_paths)

In [ ]:
# df['path'] = df['path'].apply(lambda x: np.array(x))
# df.to_csv('shortest_paths_edges.csv')

df1=pd.read_csv('shortest_paths_edges.csv')

entry_nodes = np.unique(df['entry_node'])
exit_nodes = np.unique(df['exit_node'])

N=10

shortest_entry_paths = []
for entry_node in tqdm(entry_nodes):
    for i in range(N):
        row = df[df.entry_node==entry_node].sort_values(by='path_len_pixel').iloc[i]
        shortest_entry_paths.append(row)


df1_shortest_entry = pd.DataFrame(shortest_entry_paths)


In [ ]:
df=pd.read_csv('paths.csv')

entry_nodes = np.unique(df['entry_node'])
exit_nodes = np.unique(df['exit_node'])

shortest_entry_paths = []
for entry_node in tqdm(entry_nodes):
    row = df[df.entry_node==entry_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_entry_paths.append(row)

shortest_exit_paths = []
for exit_node in tqdm(exit_nodes):
    row = df[df.exit_node==exit_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_exit_paths.append(row)

df_shortest_entry = pd.DataFrame(shortest_entry_paths)
df_shortest_exit = pd.DataFrame(shortest_exit_paths)



fig,axes = plt.subplots(1,1,figsize=(15,15))

pos = nx.get_node_attributes(g, 'pos')

for i in range(len(df_shortest_entry)):
    line = df_shortest_entry.iloc[i]
    path = line['path']
    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges,edge_color='blue',width=1)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['entry_node']],node_color='blue',node_size = 100)

for i in range(len(df_shortest_exit)):
    line = df_shortest_exit.iloc[i]
    path = list(reversed(line['path']))
    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges,edge_color='r',width=1)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['exit_node']],node_color='r',node_size = 100)

axes.imshow(img_aligned, cmap='gray')

plt.axis("on")
axes.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

axes.arrow(12, border-5, 0, 200, width=0.3, length_includes_head=True, head_width=10, head_length=10,color=(0,0,0))

axes.text(2, img_aligned.shape[0]*2/3,'hit direction',rotation=90, color=(0, 0, 0),fontsize=15)
axes.text(img_aligned.shape[1]/2-25, border/3,'entry nodes', color=(0, 0, 1),fontsize=15)
axes.text(img_aligned.shape[1]/2-25,img_aligned.shape[0] - border/3,'exit nodes', color=(1, 0, 0),fontsize=15)

# plt.savefig('shortests_paths.jpg',bbox_inches='tight')

plt.show()

# Deprecated

In [ ]:
import networkx as nx
from graph_tool.all import Graph

def nx_to_gt_with_attrs(nx_graph):
    gt_graph = Graph(directed=nx_graph.is_directed())
    vprop_name = gt_graph.new_vertex_property("string")
    eprop_weight = gt_graph.new_edge_property("float")

    nx_to_gt_v = {}
    for node in nx_graph.nodes(data=True):
        v = gt_graph.add_vertex()
        nx_to_gt_v[node[0]] = v
        if 'name' in node[1]:
            vprop_name[v] = str(node[1]['name'])

    for u, v, data in nx_graph.edges(data=True):
        e = gt_graph.add_edge(nx_to_gt_v[u], nx_to_gt_v[v])
        if 'weight' in data:
            eprop_weight[e] = float(data['weight'])

    gt_graph.vertex_properties['name'] = vprop_name
    gt_graph.edge_properties['weight'] = eprop_weight

    return gt_graph

gt_graph = nx_to_gt_with_attrs(g_weighed)

gt.graph_draw(gt_graph, vertex_text=gt_graph.vertex_index, output="wc_cv/cv/two-nodes.png")

# eweight = gt_graph.new_edge_property("weight")
eweight = gt_graph.new_edge_property("float")

vlist = gt_graph.get_vertices()
edges = gt_graph.get_edges()

paths =  list(gt.all_shortest_paths(gt_graph, source=entry_node, target=exit_node, weights=eweight))

# eweight = gt_graph.new_edge_property("weight")
eweight = gt_graph.new_edge_property("float")

vlist = gt_graph.get_vertices()
edges = gt_graph.get_edges()

paths =  list(gt.all_shortest_paths(gt_graph, source=entry_node, target=exit_node, weights=eweight))